### 1) Import functions and libraries, Initialize requirements

In [ ]:
!pip install facenet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.6 MB/s 


In [ ]:
!pip install python_telegram_bot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 519 kB 4.9 MB/s 
     |████████████████████████████████| 427 kB 51.2 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 6.0.4
    Uninstalling tornado-6.0.4:
      Successfully uninstalled tornado-6.0.4
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.2.0
    Uninstalling cachetools-5.2.0:
      Successfully uninstalled cachetools-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=6.0.4, but you have tornado 6.1 which is incompatible.


In [ ]:
from telegram.ext import *
from io import BytesIO
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training, extract_face
import torch
from PIL import Image, ImageDraw
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from google.colab import drive
import glob
from torch.autograd import Variable
from os import listdir
import copy
import seaborn as sns
sns.set_style('whitegrid')
sns.set(font_scale=1.5)
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8 
import warnings
warnings.filterwarnings("ignore")
import matplotlib.image as mpimg
import cv2
%matplotlib inline

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/datasets/Telegram_token.txt', 'r') as f:
  TOKEN = f.read()

In [ ]:
#TOKEN

### 2) Writing Bot

In [ ]:
mtcnn = MTCNN(
  image_size=160, margin=0, min_face_size=20,
  thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True)
model = torch.load('/content/drive/MyDrive/Kursovaya_4/model_final')

players = ['Antony','Aurelien Tchouameni', 'Bernardo Silva', 'Bruno Fernandes','Bukayo Saka',
 'Christopher Nkunku','Declan Rice','Dusan Vlahovic','Erling Haaland','Federico Valverde',
 'Gabriel Jesus','Gavi','Harry Kane','Jamal Musiala','Joshua Kimmich',
 'Jude Bellingham','Kevin De Bruyne','Kulian Mbappe','Lautaro Martinez','Luis Diaz',
 'Mason Mount','Mohamed Salah','Neymar','Pedri','Phil Foden',
 'Rafael Leao','Rodri','Rodrygo','Ruben Dias','Vinicius Junior']

def start(update, context):
  update.message.reply_text("Привет!  Напиши /help для знакомства с функционалом")
def help(update, context):
  update.message.reply_text("""
/start - Starts conversation
/help Shows this message
/load Loads pretrained network
  """)

def load(update, context):
  pass
  update.message.reply_text("Модель загружается...")
  
  update.message.reply_text("Готово ! Теперь можешь присылать фото (нужно обязательно сжать) !")

def handle_message(update, context):
  update.message.reply_text("Привет, можешь присылать фото (нужно обязательно сжать) !")

def handle_photo(update, context):
  #update.message.reply_text(f" На шаге 0")
  file = context.bot.get_file(update.message.photo[-1].file_id)
  #update.message.reply_text(f" На шаге 1")
  f = BytesIO(file.download_as_bytearray())
  #update.message.reply_text(f" На шаге 2")
  file_bytes = np.asarray(bytearray(f.read()), dtype=np.uint8)
  #update.message.reply_text(f" На шаге 3")
  img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  img = cv2.resize(img, (512, 512))
  #img = Image.open(file_bytes)
  #update.message.reply_text(f" На шаге 4")
  img_cropped = mtcnn(img)
  #update.message.reply_text(f" На шаге 5")
  img_probs = model(img_cropped.unsqueeze(0))
  #update.message.reply_text(f" На шаге 6")
  predicted_player = players[torch.argmax(img_probs)]
  #update.message.reply_text(f" На шаге 7")
  update.message.reply_text(f" Я думаю, что на фотографии {predicted_player}")


updater = Updater(TOKEN, use_context=True)
dp = updater.dispatcher

dp.add_handler(CommandHandler("start", start))
dp.add_handler(CommandHandler("help", help))
dp.add_handler(CommandHandler("load",load))
dp.add_handler(MessageHandler(Filters.text, handle_message))
dp.add_handler(MessageHandler(Filters.photo, handle_photo))

updater.start_polling()
updater.idle()